In [6]:
import os
from data_management import DataSource
from file_management import (SiteHierarchies, IndexFile,  FormulaFile)
from formula_tables import (FormulaTable, FormulaTableType,
                            FormulaTableTypeSimple, FormulaTableTypeCalculus,
                            FormulaTableTypeFinancial)
from content import Formulas
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)

In [13]:
a = ("John", "Charles", "Mike")
b = ("Jenny", "Christy", "Monica")

for item in zip(a, b):
    print (item)

('John', 'Jenny')
('Charles', 'Christy')
('Mike', 'Monica')
